In [4]:
#Make_SUS_Airedale_FDM_Format.ipynb
##last modified 20230309 

#Builds the BDCT SUS FDM
#Assumes that the targetdb below exists
#Needs code to build it if it doesn't

#You will need to manually amend the target dataset for this script

library(bigrquery)
library(lubridate) 

# Store the project ID
project_id = "yhcr-prd-phm-bia-core"

sourcedb <-"yhcr-prd-phm-bia-core.CB_MYSPACE_JDB"
targetdb <-'yhcr-prd-phm-bia-core.CB_STAGING_DATABASE_SUS_Airedale_FDM_Format'
lookups <-'yhcr-prd-phm-bia-core.CB_LOOKUPS'
targetdb <-gsub(' ','',targetdb)
print (sourcedb) 
print (targetdb)
print(lookups)

[1] "yhcr-prd-phm-bia-core.CB_MYSPACE_JDB"
[1] "yhcr-prd-phm-bia-core.CB_STAGING_DATABASE_SUS_Airedale_FDM_Format"


In [ ]:
# Set teh date fro this to teh last date you downloaded the data 

sql7 <-paste('update ',lookups,'.tbl_Dataset_ExtractDateRef 
set extract_date = \"2023-05-09\" where DatasetName = \"CB_STAGING_DATABASE_SUS_Airedale_FDM_Format\"', sep = "")
tb7 <-bq_project_query(project_id, sql7)
print(tb5)


In [20]:
sql4 <-paste('drop table if exists ',targetdb,'.tbl_SUS_Airedale_APC_20220601_to_Current;', sep = "")
tb4 <-bq_project_query(project_id, sql4)



sql5 <-paste('drop table if exists ',targetdb,'.tbl_SUS_Airedale_OP_20220601_to_Current;', sep = "")
tb5 <-bq_project_query(project_id, sql5)

sql6 <-paste('drop table if exists ',targetdb,'.tbl_SUS_Airedale_ECDS_20220601_to_Current;', sep = "")
tb6 <-bq_project_query(project_id, sql6)

print("tables deleted from target")

[1] "tables deleted from target"


<bq_table> yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_Dataset_ExtractDateRef


In [21]:
sql8 <-paste('create TABLE ', targetdb , '.tbl_SUS_Airedale_APC_20220601_to_Current 
as 
SELECT 
dem.person_id,
cast(left(admdate,4)||\'-\'||substring(admdate,5,2)||\'-\'|| substring(admdate,7,2)||\' 00:00:00\' as datetime) as src_SUS_Airedale_APC_20220601_to_Current_start_date,
cast(left(dischdate,4)||\'-\'||substring(dischdate,5,2)||\'-\'|| substring(dischdate,7,2)||\' 00:00:00\' as datetime)  as src_SUS_Airedale_APC_20220601_to_Current_end_date,
src.* FROM ', sourcedb,'.src_SUS_Airedale_APC_20220601_to_Current src 
left join yhcr-prd-phm-bia-core.CB_STAGING_DATABASE.src_DemoGraphics_MASTER dem
on src.digest = dem.digest ;', sep = "")
tb8 <-bq_project_query(project_id, sql8)

print("APC Built")

[1] "APC Built"


In [24]:
sql9<-paste('create TABLE ', targetdb , '.tbl_SUS_Airedale_ECDS_20220601_to_Current 
as 
SELECT 
dem.person_id,
cast(arrival_date ||\' \' || arrival_time as datetime) as src_SUS_Airedale_ECDS_20220601_to_Current_start_date,
cast(depart_date ||\' \' || depart_time as datetime) as src_SUS_Airedale_ECDS_20220601_to_Current_end_date,
src.* FROM yhcr-prd-phm-bia-core.CB_MYSPACE_JDB.src_SUS_Airedale_ECDS_20220601_to_Current src
 left join  yhcr-prd-phm-bia-core.CB_STAGING_DATABASE.src_DemoGraphics_MASTER dem
on src.digest = dem.digest
;', sep = "")
tb9 <-bq_project_query(project_id, sql9)
print("ECDS Built")

[1] "ECDS Built"


In [29]:
sql10<-paste('create TABLE ', targetdb , '.tbl_SUS_Airedale_OP_20220601_to_Current 
as 
SELECT 
dem.person_id,
cast(left(appdate,4)||\'-\'||substring(appdate,5,2)||\'-\'|| substring(appdate,7,2)||\' \'|| substring(apptime,1,2)||\':\'|| substring(apptime,3,2)||\':00\'  as datetime) as src_SUS_Airedale_OP_20220601_to_Current_start_date,
cast(left(appdate,4)||\'-\'||substring(appdate,5,2)||\'-\'|| substring(appdate,7,2)||\' \'|| substring(apptime,1,2)||\':\'|| substring(apptime,3,2)||\':00\'  as datetime) as src_SUS_Airedale_OP_20220601_to_Current_end_date,
src.* FROM yhcr-prd-phm-bia-core.CB_MYSPACE_JDB.src_SUS_Airedale_OP_20220601_to_Current src 
 left join  `yhcr-prd-phm-bia-core.CB_STAGING_DATABASE.src_DemoGraphics_MASTER` dem
on src.digest = dem.digest
;', sep = "")
tb10 <-bq_project_query(project_id, sql10)

print("OP Built")